# DeltaDTM

A global coastal digital terrain model, based on CopernicusDEM, ESA WorldCover, ICESat-2 and GEDI data. For more information, see [Pronk et al. (2024)](https://www.nature.com/articles/s41597-024-03091-9) DeltaDTM: A global coastal digital terrain model. 

In [ ]:
import fsspec
import geopandas as gpd
import hvplot.xarray
import pystac
import rioxarray
import shapely
import xarray as xr
from ipyleaflet import Map, basemaps

## Read a snapshot with the spatial extents of all tiles

Connect to the CoCliCo STAC and read the spatial extents using stac-geoparquet.

In [ ]:
from coastpy.stac.utils import read_snapshot

coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)

ddtm_collection = coclico_catalog.get_child("deltares-delta-dtm")

storage_options = ddtm_collection.extra_fields["item_assets"]["data"][
    "xarray:storage_options"
]

ddtm_extents = read_snapshot(
    ddtm_collection,
    columns=["geometry", "assets"],
    add_href=True,
    storage_options=storage_options,
)

ddtm_extents.head()

## Zoom to your area of interest

In [ ]:
m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = 15.827, -95.96
m.zoom = 15
m.layout.height = "800px"
m

In [ ]:
west, south, east, north = m.west, m.south, m.east, m.north
# Note: small little hack to ensure the notebook also works when running all cells at once
if not west:
    west, south, east, north = (
        30.28415679931641,
        31.276790311057272,
        30.630912780761722,
        31.51123970051334,
    )
roi = gpd.GeoDataFrame(
    geometry=[shapely.geometry.box(west, south, east, north)], crs=4326
)

In [ ]:
roi.explore()

## Find the tiles for your region of interest

In [ ]:
hrefs = gpd.sjoin(ddtm_extents, roi).href.to_list()
href = hrefs[0]
href

## Read data

In [ ]:
da = rioxarray.open_rasterio(href).squeeze().drop_vars("band")

In [ ]:
da.where(lambda xx: xx != xx.attrs["_FillValue"]).hvplot(
    x="x", y="y", geo=True, tiles="ESRI"
)